## Explore the titanic training set

In [32]:
import pandas as pd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [8]:
train = pd.read_csv('./data/train.csv')

In [9]:
print('Training set ( Rows , Columns ) : ', train.shape)

Training set ( Rows , Columns ) :  (891, 12)


In [10]:
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


### Missing values

* Age
* Cabin
* Embarked

In [12]:
train['Age'].fillna(train['Age'].mean(), inplace = True)

In [13]:
most_freq_port = train['Embarked'].mode()[0]
train['Embarked'].fillna(most_freq_port, inplace = True)

### Describe numeric

In [15]:
train_describe_numeric = train.describe()
train_describe_numeric

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [16]:
print('So survival rate is ')
print(train_describe_numeric.loc['mean', 'Survived'])

So survival rate is 
0.383838383838


In [17]:
train_describe_categorical = train.describe(include=['O'])
train_describe_categorical

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,891
unique,891,2,681,147,3
top,"Foreman, Mr. Benjamin Laventall",male,1601,B96 B98,S
freq,1,577,7,4,646


In [19]:
print('Most of the people on the ship were : ')
print('Sex : ', train_describe_categorical.loc['top', 'Sex'])
print('count : ', train_describe_categorical.loc['freq', 'Sex'])

Most of the people on the ship were : 
Sex :  male
count :  577


---

### Drop unnecessary features

In [21]:
columns_to_drop = ['PassengerId', 'Ticket', 'Fare', 'Cabin', 'Embarked']

train.drop(
    columns_to_drop, 
    axis = 1, 
    inplace = True)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
dtypes: float64(1), int64(4), object(2)
memory usage: 48.8+ KB


### Create new features

### Create *Family_Size* feature

In [22]:
train["Family_Size"] = train['SibSp'] + train['Parch'] + 1
train["Family_Size"].head()

0    2
1    2
2    1
3    2
4    1
Name: Family_Size, dtype: int64

### Create *Family_Type* feature

In [24]:
def getFamilyType(size):
    if size == 1:
        return 1
    elif 2 <= size <= 4:
        return 2
    else:
        return 3

train['Family_Type'] = train['Family_Size'].map(getFamilyType)
train['Family_Type'].head()

0    2
1    2
2    1
3    2
4    1
Name: Family_Type, dtype: int64

### Create *Age_Group* category

In [25]:
def getAgeGroup (age) :
    if age <= 10:
        return 1
    elif age <= 18:
        return 2
    elif age <= 28:
        return 3
    elif age <= 45:
        return 4
    elif age <= 65:
        return 5
    else:
        return 6

train[ 'Age_Group' ] = train[ 'Age' ].map(getAgeGroup)
train[ 'Age_Group' ].head()

0    3
1    4
2    3
3    4
4    4
Name: Age_Group, dtype: int64

### Encode *gender*  feature

In [33]:
train[ 'Sex' ] = labelencoder.fit_transform(train[ 'Sex' ])
train[ 'Sex' ].head()

0    1
1    0
2    0
3    0
4    1
Name: Sex, dtype: int64

----

### Visualize training set